<h1><b>Machine Learning Based Clustering and Segmentation for Navigation<b></h1>

<h3><b>Introduction</b></h3>
    <p>
    An ML based navigation algorithm that is based on several factors pertaining to neighbourhoods. That will give you the most efficient route to the desired destination, based on factors such as crime rate and population density.
    </p>
<h3><b>Prerequisites</b></h3>
<ul>
    <li>Foursquare API</li>
</ul>
<h3><b>Datasets Used</b></h3>

<h3><b>Import Statements</b></h3>

In [158]:
from dotenv import load_dotenv
from dotenv import dotenv_values
import folium
import requests
import pandas as pd 
from pandas import json_normalize

<h3><b>Foursquare API Initialization / Check</b></h3>
<h4><b>Category Codes:</b></h4>
<ul>
    <li>10000 - Arts and Entertainment</li>
    <li>11000 - Business and Professional Services</li>
    <li>12000 - Community and Government</li>
    <li>13000 - Dining and Drinking</li>
    <li>14000 - Event</li>
    <li>15000 - Health and Medicine</li>
    <li>16000 - Landmarks and Outdoors</li>
    <li>17000 - Retail</li>
    <li>18000 - Sports and Recreation</li>
    <li>19000 - Travel and Transportation</li>
</ul>

In [163]:
config = dotenv_values(".env")
url = "https://api.foursquare.com/v3/places/nearby"

headers = {"Accept": "application/json",
            "Authorization": config["API_KEY"]}

response = requests.request("GET", url, headers=headers)

def create_request(coords= None, location = None, categories = None, limit = "10"):
    """
        Important:
            - Coords and location cannot be entered together
            - Location and radius cannot be entered together

        The coords will be a list with latitude and longitude.\n 
        Location will be a city and province such as  "Oshawa, ON".\n
        The category is a string from the above codes, with a default of None.\n
        The limit parameter is a maximum of 50, with a default of 10 requests.\n

        Examples:
            - create_request(coords=[-72.848752,43.895962], limit="1")
            - create_request(coords=[-72.848752,43.895962], categories="10000", limit="2")\n
            - create_request(location=["Oshawa","ON"], limit="2")
            - create_request(location=["Oshawa","ON"], categories="10000", limit="20")
    """

    if(coords and categories == None):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) + "&radius=100000"  + "&limit=" + limit
    elif(coords and categories):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) +"&categories=" + categories + "&radius=100000" + "&limit=" + limit
    elif(location and categories == None):
        url = "https://api.foursquare.com/v3/places/search?" + "near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    elif(location and categories):
        url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    else:
        return False
    
    response = requests.request("GET", url, headers=headers)
    
    if(response.status_code == 200):
        return response.json()
    else:
        return False

<h3><b>Creating DataFrames</b></h3>

In [236]:
latitude = 41.8781
longitude = -87.6298
results = create_request(coords = [str(latitude), str(longitude)], categories="13000", limit="10")

venues = json_normalize(results['results'], max_level=3)
venues.drop(venues.columns[[0, 1, 2, 3, 5, 10,11, 12, 17, 18, 19, 20]], axis=1, inplace=True)

pd.DataFrame(venues)





,name,geocodes.main.latitude,geocodes.main.longitude,location.address,location.country,location.locality,location.neighborhood,location.postcode,location.region
0,Shaw's Crab House,41.889943,-87.627151,21 E Hubbard St,US,Chicago,[Fort Dearborn Addition],60611,IL
1,Portillo's Hot Dogs,41.893415,-87.631515,100 W Ontario St,US,Chicago,[Loop],60654,IL
2,Ricobene's,41.845656,-87.633869,252 W 26th St,US,Chicago,[Armour Square],60616,IL
3,The Aberdeen Tap,41.890242,-87.654827,440 N Aberdeen St,US,Chicago,[Carpenters Addition],60642,IL
4,Starbucks,41.893938,-87.624589,646 N Michigan Ave,US,Chicago,[Gold Coast],60611,IL
5,Portillo's,41.869586,-87.640623,520 W Taylor St,US,Chicago,[Conleys Patch],60607,IL
6,Middle Brow Bungalow,41.917734,-87.698962,2840 W Armitage Ave,US,Chicago,[Humboldt],60647,IL
7,Revolution Beer,41.942033,-87.707872,3340 N Kedzie Ave,US,Chicago,[Avondale],60618,IL
8,Sidetrack,41.943254,-87.649231,3349 N Halsted St,US,Chicago,[Graceland],60657,IL
9,Guthrie's Tavern,41.947246,-87.661856,1300 W Addison St,US,Chicago,[Lakeview],60613,IL


<h3><b>Map Creation</b></h3>


In [161]:
map_creation = folium.Map(location=[latitude, longitude], zoom_start=10)
map_creation